In [2]:
import qsharp
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as np



In [3]:
%%qsharp

open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;

operation ZZFeatureMap(qs : Qubit[], data : Double[]) : Unit {
    let numberOfQubits = Length(qs);
    if (numberOfQubits != Length(data)) {
        fail "Data length must be equal to the number of qubits.";
    }
    for i in 0..numberOfQubits-1 {
        // Apply Z rotation to each qubit
        Rz(2.0 * PI() * data[i], qs[i]);
    }
    for i in 0..numberOfQubits-1 {
        for j in i+1..numberOfQubits-1 {
            // Apply ZZ interaction between each pair of qubits
            let theta = 2.0 * PI() * data[i] * data[j];
            CNOT(qs[i], qs[j]);
            Rz(theta, qs[j]);
            CNOT(qs[i], qs[j]);
        }
    }
}


operation RealAmplitudes(qubits : Qubit[], parameters : Double[]) : Unit {
    // Apply layer of rotation gates
    for idx in IndexRange(qubits) {
        Ry(parameters[idx], qubits[idx]);
    }

    // Apply a layer of CNOT gates
    for idx in 0 .. Length(qubits)-2 {
        for i in idx+1 .. Length(qubits)-1{
            CNOT(qubits[idx], qubits[i]);
        }
    }


    // Apply another layer of rotation gates
    for idx in IndexRange(qubits) {
        Ry(parameters[idx + Length(qubits)], qubits[idx]);
    }
}

operation quantumVariationalClassifier (qubits: Qubit[], x : Double[], θ : Double[]) : Unit {
    ZZFeatureMap(qubits, x);
    RealAmplitudes(qubits, θ);
}



In [ ]:
def cost(x):
    return quantumVariationalClassifier(x)

opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
init_params = np.array([0.011, 0.012, 0.013, 0.014], requires_grad=True)
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))
